In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/reuters_health.txt'
dest_url = './run_data/latimes_scrapt_v2.csv'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [ ]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

In [ ]:
# -------------------------------------------------
frame.shape

In [ ]:
frame.info()

In [ ]:
# -------------------------------------------------
frame.loc[:,'title'][0]

In [ ]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(split_txt_form_url)

In [ ]:
frame.head()

In [ ]:
frame.info()

In [ ]:
# -------------------------------------------------
frame_urls = frame['url'].values

In [ ]:
# -------------------------------------------------
# Run for this jupyter notebook
title_attempt = True
for key, url in enumerate(frame_urls):
    try:
        # Start the connection
        res = requests.get(url)        

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
            print(url)
            
            # Initialize XPath
            xpath_selector = Selector(text=res.text)
            
            list_content = dict()
            
            # Scrapt Text with XPath
            contents = xpath_selector.xpath('//div[@class=\"StandardArticleBody_body\"]/p/text()').extract()
            contents = join_txt(contents)
            print(len(contents))
#             # Tags
#             tags = xpath_selector.xpath('').extract()     
#             print(len(tags))
            # Author
            author = xpath_selector.xpath('//div[@class=\"BylineBar_byline\"]/span/a/text()').extract()
            if len(author):
                author = author[0]
            else:
                author = ''
            print(len(author))
            list_content.update({'contents': contents,'tags': '','author': author})
            
            if title_attempt:
                print("TItle: ", title_attempt)
                df_temp = pd.DataFrame(columns = list(list_content.keys()))
                frame = pd.concat([frame, df_temp])
                print(frame.columns)                
                title_attempt = False
                
            for k_, v_ in list_content.items():
                print(k_," : " ,len(v_))
                frame.loc[key,k_] = v_
            print("\n")

        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))
    except:
        pass
    

In [ ]:
# -------------------------------------------------
frame.info()

In [ ]:
frame.head()

In [ ]:
frame.to_csv(dest_url, sep='|', encoding='utf-8',index=False)

In [2]:
import unicodedata
# If End of jupyter-notebook, reset the "frame" Dataframe as 4 columns
frame = pd.read_csv(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame_content = pd.read_csv(dest_url, sep='|', encoding="utf-8")
### Writing content into text file
### ================================
file_names = frame['tweetid'].values

for key, name in enumerate(file_names):
    
    # Construct the file name
    located_file = './training/' + str(name) + '.txt'
#     success_date = './date/success_date.txt'
    reg_special = r"([^a-zA-Z0-9\s\']+)"
    reg_dspace = r"( {2,})"   # r"(\s{2,})"
    reg_html = r"(<[^>]+>)"
    reg_newlines = r"([\r|\n|\r\n]+)"
    
    try:
    
        # Writing on file
        contents = str(frame_content['contents'].iloc[key])
        print(key, ' : ' ,len(contents))
        if len(contents) > 10:
            # Open file for writing
            f= open(located_file,"w+")
            
            # Clean content a bit
            contents = re.sub(reg_html," ",contents)
            contents = unicodedata.normalize('NFKD', contents).encode('ascii', 'ignore').decode('ascii')
            contents = re.sub(reg_special," ", contents)
            contents = re.sub(reg_newlines," ",contents)
            contents = re.sub(reg_dspace," ",contents)
            
            # Start writing content to file.
            f.write(contents)
            
            # close file connection
            f.close()
            
            #Write success load date
#             f1 = open(success_date, "a")
#             f1.write(frame['date'].iloc[key])
#             f1.close()
        else:
            print("[Out] key: ", key) 
    except:
        pass

### ================================

0  :  5405
1  :  2135
2  :  5918
3  :  3079
4  :  2056
5  :  2776
6  :  4668
7  :  3
[Out] key:  7
8  :  4041
9  :  3502
10  :  3502
11  :  4372
12  :  3502
13  :  3502
14  :  3502
15  :  4686
16  :  4686
17  :  7289
18  :  105
19  :  7289
20  :  4686
21  :  7289
22  :  7289
23  :  3425
24  :  3425
25  :  5737
26  :  2863
27  :  4922
28  :  4806
29  :  4922
30  :  2066
31  :  3849
32  :  2224
33  :  1681
34  :  1681
35  :  1681
36  :  3431
37  :  4922
38  :  3
[Out] key:  38
39  :  3
[Out] key:  39
40  :  3
[Out] key:  40
41  :  3
[Out] key:  41
42  :  3
[Out] key:  42
43  :  3
[Out] key:  43
44  :  3
[Out] key:  44
45  :  3
[Out] key:  45
46  :  3111
47  :  4040
48  :  3111
49  :  3431
50  :  3431
51  :  3111
52  :  3911
53  :  3911
54  :  3911
55  :  3911
56  :  3911
57  :  3911
58  :  4425
59  :  4425
60  :  4425
61  :  4425
62  :  5324
63  :  6479
64  :  4268
65  :  3170
66  :  3170
67  :  6479
68  :  5787
69  :  3129
70  :  2277
71  :  2228
72  :  2636
73  :  5186
74  :  1571
75  

620  :  4009
621  :  4009
622  :  1208
623  :  281
624  :  3275
625  :  3947
626  :  2175
627  :  2710
628  :  3947
629  :  3947
630  :  2985
631  :  2447
632  :  2636
633  :  2636
634  :  4807
635  :  1052
636  :  4041
637  :  3959
638  :  2575
639  :  2887
640  :  2449
641  :  1464
642  :  1464
643  :  4215
644  :  4215
645  :  3029
646  :  3752
647  :  3752
648  :  6868
649  :  2534
650  :  6119
651  :  3873
652  :  5102
653  :  6119
654  :  5102
655  :  6119
656  :  2912
657  :  3159
658  :  3564
659  :  3
[Out] key:  659
660  :  8913
661  :  4032
662  :  1904
663  :  1724
664  :  6339
665  :  6339
666  :  4288
667  :  4288
668  :  3166
669  :  4429
670  :  10018
671  :  4561
672  :  1061
673  :  3319
674  :  2236
675  :  1937
676  :  3683
677  :  4205
678  :  2231
679  :  6503
680  :  3696
681  :  3696
682  :  3621
683  :  3444
684  :  1567
685  :  1567
686  :  4722
687  :  3
[Out] key:  687
688  :  1435
689  :  6270
690  :  5146
691  :  5146
692  :  3129
693  :  3284
694  :  2767

1182  :  4645
1183  :  2399
1184  :  3630
1185  :  3630
1186  :  3
[Out] key:  1186
1187  :  2333
1188  :  4772
1189  :  3990
1190  :  3
[Out] key:  1190
1191  :  2245
1192  :  3
[Out] key:  1192
1193  :  3
[Out] key:  1193
1194  :  4393
1195  :  4393
1196  :  2853
1197  :  4464
1198  :  4063
1199  :  4063
1200  :  4077
1201  :  3924
1202  :  4077
1203  :  3
[Out] key:  1203
1204  :  5113
1205  :  3
[Out] key:  1205
1206  :  5978
1207  :  2239
1208  :  5978
1209  :  3
[Out] key:  1209
1210  :  3207
1211  :  2247
1212  :  2100
1213  :  2100
1214  :  3
[Out] key:  1214
1215  :  3961
1216  :  3201
1217  :  3173
1218  :  5928
1219  :  3961
1220  :  3173
1221  :  3941
1222  :  4170
1223  :  4170
1224  :  2868
1225  :  5012
1226  :  5433
1227  :  4520
1228  :  3
[Out] key:  1228
1229  :  6243
1230  :  3263
1231  :  3
[Out] key:  1231
1232  :  2706
1233  :  5037
1234  :  2634
1235  :  2545
1236  :  3317
1237  :  2195
1238  :  3368
1239  :  2634
1240  :  5144
1241  :  5144
1242  :  5144
1243  

1678  :  3
[Out] key:  1678
1679  :  3
[Out] key:  1679
1680  :  3287
1681  :  3
[Out] key:  1681
1682  :  3
[Out] key:  1682
1683  :  3
[Out] key:  1683
1684  :  3
[Out] key:  1684
1685  :  3
[Out] key:  1685
1686  :  1043
1687  :  3
[Out] key:  1687
1688  :  3
[Out] key:  1688
1689  :  3
[Out] key:  1689
1690  :  3
[Out] key:  1690
1691  :  3
[Out] key:  1691
1692  :  3
[Out] key:  1692
1693  :  3
[Out] key:  1693
1694  :  3
[Out] key:  1694
1695  :  3
[Out] key:  1695
1696  :  3
[Out] key:  1696
1697  :  3
[Out] key:  1697
1698  :  3
[Out] key:  1698
1699  :  3
[Out] key:  1699
1700  :  3
[Out] key:  1700
1701  :  3
[Out] key:  1701
1702  :  2569
1703  :  3
[Out] key:  1703
1704  :  3778
1705  :  3161
1706  :  3
[Out] key:  1706
1707  :  3778
1708  :  3
[Out] key:  1708
1709  :  3
[Out] key:  1709
1710  :  3
[Out] key:  1710
1711  :  3
[Out] key:  1711
1712  :  3
[Out] key:  1712
1713  :  3
[Out] key:  1713
1714  :  3
[Out] key:  1714
1715  :  3
[Out] key:  1715
1716  :  3
[Out] key

2085  :  3
[Out] key:  2085
2086  :  3
[Out] key:  2086
2087  :  3
[Out] key:  2087
2088  :  3
[Out] key:  2088
2089  :  3
[Out] key:  2089
2090  :  3
[Out] key:  2090
2091  :  3
[Out] key:  2091
2092  :  3
[Out] key:  2092
2093  :  3
[Out] key:  2093
2094  :  3
[Out] key:  2094
2095  :  3
[Out] key:  2095
2096  :  3
[Out] key:  2096
2097  :  3
[Out] key:  2097
2098  :  3
[Out] key:  2098
2099  :  3
[Out] key:  2099
2100  :  3
[Out] key:  2100
2101  :  3
[Out] key:  2101
2102  :  3
[Out] key:  2102
2103  :  3
[Out] key:  2103
2104  :  3
[Out] key:  2104
2105  :  3
[Out] key:  2105
2106  :  3
[Out] key:  2106
2107  :  3
[Out] key:  2107
2108  :  3
[Out] key:  2108
2109  :  3
[Out] key:  2109
2110  :  3
[Out] key:  2110
2111  :  3
[Out] key:  2111
2112  :  3
[Out] key:  2112
2113  :  3
[Out] key:  2113
2114  :  3
[Out] key:  2114
2115  :  3
[Out] key:  2115
2116  :  3
[Out] key:  2116
2117  :  3
[Out] key:  2117
2118  :  3
[Out] key:  2118
2119  :  3
[Out] key:  2119
2120  :  3
[Out] key

[Out] key:  2380
2381  :  3
[Out] key:  2381
2382  :  3
[Out] key:  2382
2383  :  3
[Out] key:  2383
2384  :  3
[Out] key:  2384
2385  :  3
[Out] key:  2385
2386  :  3
[Out] key:  2386
2387  :  3
[Out] key:  2387
2388  :  3
[Out] key:  2388
2389  :  3
[Out] key:  2389
2390  :  3
[Out] key:  2390
2391  :  3
[Out] key:  2391
2392  :  3
[Out] key:  2392
2393  :  3
[Out] key:  2393
2394  :  3
[Out] key:  2394
2395  :  3
[Out] key:  2395
2396  :  3
[Out] key:  2396
2397  :  3
[Out] key:  2397
2398  :  3
[Out] key:  2398
2399  :  3
[Out] key:  2399
2400  :  3
[Out] key:  2400
2401  :  3
[Out] key:  2401
2402  :  3
[Out] key:  2402
2403  :  3
[Out] key:  2403
2404  :  3
[Out] key:  2404
2405  :  3
[Out] key:  2405
2406  :  3
[Out] key:  2406
2407  :  3
[Out] key:  2407
2408  :  3
[Out] key:  2408
2409  :  3
[Out] key:  2409
2410  :  3
[Out] key:  2410
2411  :  3
[Out] key:  2411
2412  :  3
[Out] key:  2412
2413  :  3
[Out] key:  2413
2414  :  3
[Out] key:  2414
2415  :  3
[Out] key:  2415
241

2739  :  3
[Out] key:  2739
2740  :  3
[Out] key:  2740
2741  :  3
[Out] key:  2741
2742  :  3
[Out] key:  2742
2743  :  2897
2744  :  1345
2745  :  3
[Out] key:  2745
2746  :  3318
2747  :  1021
2748  :  3
[Out] key:  2748
2749  :  3
[Out] key:  2749
2750  :  3
[Out] key:  2750
2751  :  3
[Out] key:  2751
2752  :  3
[Out] key:  2752
2753  :  3
[Out] key:  2753
2754  :  3
[Out] key:  2754
2755  :  3
[Out] key:  2755
2756  :  3
[Out] key:  2756
2757  :  3
[Out] key:  2757
2758  :  3
[Out] key:  2758
2759  :  3
[Out] key:  2759
2760  :  3
[Out] key:  2760
2761  :  3
[Out] key:  2761
2762  :  3
[Out] key:  2762
2763  :  3
[Out] key:  2763
2764  :  3
[Out] key:  2764
2765  :  3800
2766  :  5026
2767  :  3
[Out] key:  2767
2768  :  3
[Out] key:  2768
2769  :  3
[Out] key:  2769
2770  :  3
[Out] key:  2770
2771  :  3
[Out] key:  2771
2772  :  3
[Out] key:  2772
2773  :  3
[Out] key:  2773
2774  :  3
[Out] key:  2774
2775  :  3
[Out] key:  2775
2776  :  3
[Out] key:  2776
2777  :  3
[Out] key

3233  :  3
[Out] key:  3233
3234  :  3
[Out] key:  3234
3235  :  3
[Out] key:  3235
3236  :  3
[Out] key:  3236
3237  :  3
[Out] key:  3237
3238  :  3
[Out] key:  3238
3239  :  3
[Out] key:  3239
3240  :  3
[Out] key:  3240
3241  :  3
[Out] key:  3241
3242  :  3
[Out] key:  3242
3243  :  3
[Out] key:  3243
3244  :  3
[Out] key:  3244
3245  :  3
[Out] key:  3245
3246  :  3
[Out] key:  3246
3247  :  3
[Out] key:  3247
3248  :  3
[Out] key:  3248
3249  :  3
[Out] key:  3249
3250  :  3
[Out] key:  3250
3251  :  3
[Out] key:  3251
3252  :  3
[Out] key:  3252
3253  :  3
[Out] key:  3253
3254  :  3
[Out] key:  3254
3255  :  3
[Out] key:  3255
3256  :  3
[Out] key:  3256
3257  :  3
[Out] key:  3257
3258  :  3
[Out] key:  3258
3259  :  3
[Out] key:  3259
3260  :  3
[Out] key:  3260
3261  :  3
[Out] key:  3261
3262  :  3
[Out] key:  3262
3263  :  3
[Out] key:  3263
3264  :  3
[Out] key:  3264
3265  :  3
[Out] key:  3265
3266  :  3
[Out] key:  3266
3267  :  3
[Out] key:  3267
3268  :  3
[Out] key

3605  :  3
[Out] key:  3605
3606  :  3
[Out] key:  3606
3607  :  3
[Out] key:  3607
3608  :  3
[Out] key:  3608
3609  :  3
[Out] key:  3609
3610  :  3
[Out] key:  3610
3611  :  3
[Out] key:  3611
3612  :  3506
3613  :  3
[Out] key:  3613
3614  :  3
[Out] key:  3614
3615  :  3
[Out] key:  3615
3616  :  3
[Out] key:  3616
3617  :  3
[Out] key:  3617
3618  :  3
[Out] key:  3618
3619  :  3
[Out] key:  3619
3620  :  3
[Out] key:  3620
3621  :  3
[Out] key:  3621
3622  :  3
[Out] key:  3622
3623  :  3
[Out] key:  3623
3624  :  3
[Out] key:  3624
3625  :  3
[Out] key:  3625
3626  :  5180
3627  :  3
[Out] key:  3627
3628  :  3
[Out] key:  3628
3629  :  3
[Out] key:  3629
3630  :  3
[Out] key:  3630
3631  :  3
[Out] key:  3631
3632  :  2135
3633  :  3
[Out] key:  3633
3634  :  3
[Out] key:  3634
3635  :  3
[Out] key:  3635
3636  :  3
[Out] key:  3636
3637  :  3
[Out] key:  3637
3638  :  3
[Out] key:  3638
3639  :  3
[Out] key:  3639
3640  :  3
[Out] key:  3640
3641  :  3
[Out] key:  3641
3642  

3932  :  3
[Out] key:  3932
3933  :  3
[Out] key:  3933
3934  :  3
[Out] key:  3934
3935  :  3
[Out] key:  3935
3936  :  3
[Out] key:  3936
3937  :  3
[Out] key:  3937
3938  :  3
[Out] key:  3938
3939  :  3
[Out] key:  3939
3940  :  3
[Out] key:  3940
3941  :  3
[Out] key:  3941
3942  :  3
[Out] key:  3942
3943  :  3
[Out] key:  3943
3944  :  3
[Out] key:  3944
3945  :  3
[Out] key:  3945
3946  :  5821
3947  :  3
[Out] key:  3947
3948  :  3
[Out] key:  3948
3949  :  3
[Out] key:  3949
3950  :  3
[Out] key:  3950
3951  :  3
[Out] key:  3951
3952  :  3
[Out] key:  3952
3953  :  3
[Out] key:  3953
3954  :  3
[Out] key:  3954
3955  :  3
[Out] key:  3955
3956  :  3
[Out] key:  3956
3957  :  3
[Out] key:  3957
3958  :  3
[Out] key:  3958
3959  :  3
[Out] key:  3959
3960  :  3
[Out] key:  3960
3961  :  3
[Out] key:  3961
3962  :  3
[Out] key:  3962
3963  :  3
[Out] key:  3963
3964  :  3168
3965  :  3
[Out] key:  3965
3966  :  4318
3967  :  3
[Out] key:  3967
3968  :  3
[Out] key:  3968
3969  